In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv(r"C:\Users\HP\Desktop\Policy lense intern\archive\IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [6]:
data.shape


(50000, 2)

In [7]:
data["sentiment"].value_counts()


positive    25000
negative    25000
Name: sentiment, dtype: int64

label encoder 

In [8]:
# positive -> 1
# negative -> 0
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [9]:
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


# LSTM -> LONG SHORT TERM MEMORY

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [12]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)


In [13]:
test_data.shape



(10000, 2)

In [14]:
train_data.shape


(40000, 2)

In [15]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [16]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [17]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]])

In [18]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]])

In [19]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [20]:
Y_train



39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


# LSTM MODEL BUILDING

In [31]:
model = Sequential()
model.add(Embedding(input_dim =5000, output_dim = 128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

In [32]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])


In [34]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)


Epoch 1/5
500/500 [==============================] - 964s 2s/step - loss: 0.4008 - accuracy: 0.8193 - val_loss: 0.3195 - val_accuracy: 0.8651
Epoch 2/5
500/500 [==============================] - 885s 2s/step - loss: 0.2642 - accuracy: 0.8949 - val_loss: 0.3208 - val_accuracy: 0.8611
Epoch 3/5
500/500 [==============================] - 959s 2s/step - loss: 0.2278 - accuracy: 0.9114 - val_loss: 0.3347 - val_accuracy: 0.8724
Epoch 4/5
500/500 [==============================] - 811s 2s/step - loss: 0.1914 - accuracy: 0.9267 - val_loss: 0.3290 - val_accuracy: 0.8754
Epoch 5/5
500/500 [==============================] - 801s 2s/step - loss: 0.1677 - accuracy: 0.9363 - val_loss: 0.3357 - val_accuracy: 0.8730


In [36]:
model.save("model.h5")


In [62]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [63]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [37]:
loss, accuracy = model.evaluate(X_test, Y_test)


313/313 [==============================] - 29s 90ms/step - loss: 0.3135 - accuracy: 0.8788


In [38]:
print(loss)


0.3134870231151581


In [39]:
print(accuracy)


0.8787999749183655


Testing of model 

In [40]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [41]:
predictive_system("This movie was fantastic and amazing")


1/1 [==============================] - 1s 786ms/step


'positive'

In [42]:
predictive_system("Overall long and slow")




1/1 [==============================] - 0s 69ms/step


'negative'

In [43]:
predictive_system("Overall long and slow")


1/1 [==============================] - 0s 57ms/step


'negative'

In [44]:
predictive_system("A trilling adventure with stunning visual")


1/1 [==============================] - 0s 74ms/step


'positive'

In [ ]:
# Logistic Regression

In [52]:
from  sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [53]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

model Evaluation

In [54]:
X_train_prediction =model.predict(X_train)
traning_Data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [55]:
print("Accuracy train data:",traning_Data_accuracy)

Accuracy train data: 0.53135


In [56]:
X_test_prediction =model.predict(X_test)
test_Data_accuracy = accuracy_score(Y_test,X_test_prediction)

In [57]:
print("Accuracy test data :",test_Data_accuracy)

Accuracy test data : 0.5055


Logistic regression is not that suitable for sentiment analysis of IMDB movie review Its Accuracy is aout 50% only 